In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
# Cargar el dataset procesado
df = pd.read_csv(r"C:\Users\franc\Desktop\ML_IoT\df_final_for_training.csv", sep=',')
df.shape

In [ ]:
df.columns

In [ ]:
# Separar variables predictoras (X) y la variable objetivo (y)
X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size=0.2, random_state=42)

In [ ]:
X_test.columns

#### Función para entrenar modelos y evaluar rendimiento

In [ ]:
def train_and_evaluate(model, model_name):
    model.fit(X_train,y_train )
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    return{
        'Modelo': model_name,
        'RMSE': rmse,
        'R2' : r2,
        'MAE' : mae
    }

#### Inicializar modelos

In [ ]:
models = [
    (RandomForestRegressor(n_estimators=100, random_state=42), "Random Forest"),
    (XGBRegressor(n_estimators=100, random_state=42), "XGBoost"),
    (LGBMRegressor(n_estimators=100, random_state=42), "LightGBM")
]

### Evaluar cada modelo

In [ ]:
results=[]
for model, name in models:
    results.append(train_and_evaluate(model, name))

# Calcular el Accuracy como el porcentaje de varianza explicada (R² en porcentaje)

    
results_df = pd.DataFrame(results)

results_df["Accuracy (%)"] = results_df["R2"] * 100
results_df


#### Visualizar la comparación del rendimiento

In [ ]:
# Visualizar comparación de rendimiento
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.barplot(x='Modelo', y='RMSE', data=results_df, palette='coolwarm', ax=axes[0])
axes[0].set_title('Comparación de RMSE entre Modelos')
axes[0].set_ylabel('RMSE')

sns.barplot(x='Modelo', y='R2', data=results_df, palette='coolwarm', ax=axes[1])
axes[1].set_title('Comparación de R² entre Modelos')
axes[1].set_ylabel('R²')

sns.barplot(x='Modelo', y='MAE', data=results_df, palette='coolwarm', ax=axes[2])
axes[2].set_title('Comparación de MAE entre Modelos')
axes[2].set_ylabel('MAE')

plt.tight_layout()
plt.show()

 #### MODELO SELECCIONADO XGBoost

 Antes de ajustar hiperparametros consideramos añadir dos variables más que pueden aportar información real y mejorar el rendimiento del modelo.

- TotRmsAbvGrd -> Todas las viviendas tienen habitaciones, esta variable ayuda a capturar el tamaño real de la casa.
- KitchenQual -> Todas las viviendas tienen cocina, y su calidad afecta directamente el valor de venta.

In [ ]:
# Cargar el dataset original con todas las variables
df_full = pd.read_csv(r"C:\Users\franc\Desktop\ML_IoT\df_final.csv", sep=',')

# Variables que queremos mantener (las 9 originales + las 2 nuevas)
selected_vars = [
    'OverallQual', 'GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'BsmtFinSF1',
    '2ndFlrSF', 'GarageCars', 'YearBuilt', 'GarageArea', 'TotRmsAbvGrd', 'KitchenQual', 'SalePrice'
]


df_updated = df_full[selected_vars]
df_updated.dtypes

####  Identificar si hay variables categóricas para aplicar One-Hot Encoding

In [ ]:
categorical_vars = ['KitchenQual']
df_updated = pd.get_dummies(df_updated, columns=categorical_vars, drop_first=True)

#### Separar variables predictoras y la variable objetivo 

In [ ]:
X_updated = df_updated.drop(columns=["SalePrice"])
y_updated = df_updated["SalePrice"]

In [ ]:
X_updated.columns

#### Dividir en conjunto de entrenamiento y prueba (80-20)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_updated, y_updated, test_size=0.2, random_state=42)

#### Entrnamiento XGBoost con las nuevas variables

In [ ]:
xgb_model_updated = XGBRegressor(n_estimators=100, random_state=42)
xgb_model_updated.fit(X_train, y_train)

#### Predecir sobre el conjunto de prueba

In [ ]:
y_pred_updated = xgb_model_updated.predict(X_test)

#### Evaluar el modelo con las nuevas variables

In [ ]:
rmse_updated = np.sqrt(mean_squared_error(y_test, y_pred_updated))
r2_updated = r2_score(y_test, y_pred_updated)
mae_updated = mean_absolute_error(y_test, y_pred_updated)

results_updated = pd.DataFrame({
    "Modelo": ["XGBoost Mejorado"],
    "RMSE": [rmse_updated],
    "R2": [r2_updated],
    "MAE": [mae_updated]
})

results_updated["Accuracy (%)"] = results_updated["R2"] * 100

results_updated 

#### Resultado:

El rendimiento ha bajado:

| Modelo               | RMSE ↓ | R² ↑  | MAE ↓  | Accuracy ↑ |
|----------------------|--------|-------|--------|------------|
| XGBoost Mejorado     | 30,208 | 0.8810| 20,121 | 88.10%     |
| XGBoost              | 29,460 | 0.8868| 19,685 | 88.68%   

Próximos pasos ajustar hiperparametros de el modelo XGBoost, que es el que mejores resultados a obtenido.  |

In [ ]:
df_final_for_training = df.copy()

# Separar variables predictoras (X) y la variable objetivo (y)
X_train_final = df_final_for_training.drop(columns=["SalePrice"])
y_train_final = df_final_for_training["SalePrice"]

# Definir un espacio de búsqueda optimizado para evitar problemas de carga
param_grid_reduced = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1]
}

# Inicializar el modelo base XGBoost
xgb_base = XGBRegressor(random_state=42)

# Configurar la búsqueda aleatoria de hiperparámetros
random_search_final = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_grid_reduced,
    n_iter=5,  
    scoring='r2',
    cv=3,  
    verbose=1,
    random_state=42,
    n_jobs=-1  
)

# Entrenar la búsqueda aleatoria en el dataset correcto
random_search_final.fit(X_train_final, y_train_final)

# Obtener los mejores hiperparámetros
best_params_final = random_search_final.best_params_
best_score_final = random_search_final.best_score_

print("Mejores parametros: ", best_params_final)
print("Mejor puntuación final: ", best_score_final)

#### Resultado:

El ajuste de hiperparametros empeora el rendimiento del modelo

### Solo ajustes hiperparámetros clave